<h2>Редукция конечного акцептора</h2>

Минимальный автомат — это автомат, имеющий наименьшее возможное количество состояний и реализующий заданную функцию выходов. Задача минимизации автомата сводится к поиску его минимальной формы.
<br><br>

Пусть А - конечный акцептор:<br>
A = $(\sum, S, s_0 \in S, F, \sigma:S \times \sum \to S)$
<br>
<br>Если в ДКА существуют два эквивалентных состояния, то при их объединении мы получим эквивалентный ДКА, так как распознаваемый язык не изменится. Основная идея минимизации состоит в разбиении множества состояний на классы эквивалентности, полученные классы и будут состояниями минимизированного ДКА.

Далее показан псевдокод программы, которая реализовывает алгоритм Хопкрофта:

In [ ]:
function findEquivalenceClasses(Q, F, δ): vector 
   P←{F, Q∖F}
   S←∅
   for c∈Σ
     push ⟨F, c⟩, ⟨Q∖F, c⟩ into S
   while S≠∅
     ⟨C, a⟩ ← pop from S
     Inverse←{r | r∈Q, δ(r,a)∈C}
     T′←{R | R∈P, R∩Inverse≠∅} // находим классы, из состояний которых есть ребро в состояния сплиттера 
     for R in T′ // перебираем только классы входящие в T′
       R1,R2← split(R, C, a)
       if R1≠∅ and R2≠∅
        replace R in P with R1 and R2
        if ⟨R, c⟩ in S
          remove ⟨R,c⟩ from S
          push ⟨R1,c⟩ into S
          push ⟨R2,c⟩ into S
        else
           if |P[R1]|⩽|P[R2]|
             push ⟨R1,c⟩ into S
           else
             push ⟨R2,c⟩ into S
   return P

Код программы, которая с помощью алгоритма Хопкрофта проводит редукцию конечного акцептора:

In [6]:
from typing import Dict

def hopcroft_minimization(sigma =None, 
                          instances = None, 
                          s0 = None, 
                          final_instances = None, 
                          omega:Dict[object, object]=None) -> set:
    
    if not isinstance(sigma, set):
        raise TypeError('Incorrect type for Sigma. Use set().')
            
    if not isinstance(instances, set):
        raise TypeError('Incorrect type for Instances. Use set().')
            
    if s0 not in instances:
        raise ValueError('Incorrect Default Instance. You have to use the one from the Instance Set.')
            
    if not isinstance(final_instances, set):
        raise TypeError('Incorrect type for Final Instances. Use set().')
            
    for instance in final_instances:
        if instance not in instances:
            raise ValueError(f'{instance} from final_instances is not in instances.')
            
    if not isinstance(omega, dict):
        raise TypeError('Incorrect type for Omega.')
            
    s0_in_omega = False
            
    for key, value in omega.items():
            
        if not isinstance(key, tuple):
            raise TypeError(f'{key} (type {type(key)}) is not of type tuple.')
                
        if len(key) != 2:
            raise TypeError(f'Keys in omega should be tuple with size 2, not {len(key)} ({key}).')
                
        s, e = key
            
        if s not in instances:
            raise ValueError(f's in {key} is not in instances.')
                
        if e not in sigma:
            raise ValueError(f'e in {key} is not in sigma.')
                
        if value not in instances:
            raise ValueError(f'{value} is not in sigma.')
                
        s0_in_omega = (s == s0) or s0_in_omega

    if not s0_in_omega:
        raise ValueError('s0 is not available in omega.')

    p = [final_instances, instances.difference(final_instances)]

    w = [final_instances, instances.difference(final_instances)]

    while len(w) != 0:
        a = w.pop()

        for c in sigma:
            x = {state for state in instances if omega[(state, c)] in a}

            for y in p:

                y_diff_x = y.difference(x)

                y_intersect_x = y.intersection(x)

                if len(y_diff_x) != 0 and len(y_intersect_x) != 0:

                    p.remove(y)
                    p.extend([y_diff_x, y_intersect_x])

                    if y in w:
                        w.remove(y)
                        w.extend([y_diff_x, y_intersect_x])
                    else:
                        if len(y_intersect_x) <= len(y_diff_x):
                            w.append(y_intersect_x)
                        else:
                            w.append(y_diff_x)

    return p